<a href="https://colab.research.google.com/github/Himanshu069/3D-AI-learning/blob/main/Point_Cloud_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install open3d
!pip install plotly
!pip install trimesh

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.6 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.1/700.1 kB 16.3 MB/s eta 0:00:00


In [2]:
#generating dataset
import trimesh
import random
import os
import numpy as np

# Create a directory to save the generated shapes
output_dir = "random_shapes_dataset"
os.makedirs(output_dir, exist_ok=True)
labels=[]

# Function to generate a random shape and save it as an OBJ file
def generate_random_shape(index):
    # Randomly choose a shape type: 0 = cube, 1 = sphere, 2 = cylinder, 3 = cone
    shape_type = random.choice(["cube", "sphere", "cylinder", "cone"])

    # Generate random parameters for the shape
    scale = random.uniform(0.5, 2.0)  # Random scale
    if shape_type == "cube":
        labels.append(0)
        shape = trimesh.creation.box(extents=(scale, scale, scale))
    elif shape_type == "sphere":
        labels.append(1)
        radius = random.uniform(0.5, 2.0)
        shape = trimesh.creation.icosphere(subdivisions=3, radius=radius)
    elif shape_type == "cylinder":
        labels.append(2)
        radius = random.uniform(0.5, 1.5)
        height = random.uniform(0.5, 2.0)
        shape = trimesh.creation.cylinder(radius=radius, height=height)
    elif shape_type == "cone":
        labels.append(3)
        radius = random.uniform(0.5, 1.5)
        height = random.uniform(1.0, 3.0)
        shape = trimesh.creation.cone(radius=radius, height=height)

    # Apply random rotation and translation
    shape.apply_transform(trimesh.transformations.random_rotation_matrix())
    shape.apply_translation([random.uniform(-5, 5), random.uniform(-5, 5), random.uniform(-5, 5)])

    # Save the shape as an OBJ file
    filename = f"{shape_type}_{index}.obj"
    filepath = os.path.join(output_dir, filename)
    shape.export(filepath)

# Generate N random shapes
N = 10000  # Number of shapes to generate
for i in range(N):
    generate_random_shape(i)

print(f"Generated {N} random shapes in '{output_dir}' directory.")

Generated 10000 random shapes in 'random_shapes_dataset' directory.


In [3]:
def obj_to_pointcloud(filepath, num_points = 1024):
                      mesh = trimesh.load(filepath)

                      points, _ = trimesh.sample.sample_surface(mesh, num_points)

                      return points

pointclouds=[]


for obj_file in os.listdir(output_dir):
    if obj_file.endswith(".obj"):
      pointcloud = obj_to_pointcloud(os.path.join(output_dir, obj_file))
      pointclouds.append(pointcloud)

#print(pointclouds[990])
pointclouds = np.array(pointclouds)
print(pointclouds)

[[[ 2.76986793  0.79246012 -3.04681305]
  [ 2.9057342   0.69973828 -4.32626751]
  [ 2.80691134  0.83077167 -4.14942605]
  ...
  [ 2.75912585  0.62911375 -3.17041415]
  [ 2.91760078  1.30400978 -3.35727644]
  [ 3.83969253  0.48628075 -3.62977647]]

 [[ 2.74976549  4.37843532 -2.92942363]
  [ 1.39749327  3.28681377 -2.61490782]
  [ 1.96717046  3.26771273 -3.19064536]
  ...
  [ 2.63238049  3.71580968 -2.79299901]
  [ 2.24288072  3.75226041 -3.56767546]
  [ 2.72586557  4.39722953 -2.92754609]]

 [[ 5.63423858 -1.58644902  1.82359787]
  [ 6.1102929  -1.40292457  0.07481978]
  [ 5.43897128 -2.48340781  1.52566614]
  ...
  [ 4.84479916 -0.95273272 -0.01857252]
  [ 4.29715398 -2.84346282  0.37039393]
  [ 5.3543862  -0.86561276  0.95961827]]

 ...

 [[-2.15811312 -4.84588423  1.51546416]
  [-2.75494168 -3.37550033  0.83645908]
  [-1.65987302 -4.66776251  1.98291017]
  ...
  [-2.68218668 -2.29325177  2.50374344]
  [-1.62182409 -4.40255379  3.01434271]
  [-2.73171893 -3.2424352   0.859297  ]]

 [

In [4]:
#converting to pytorch
import torch
from torch.utils.data import Dataset, DataLoader
class PointCloudDataset(Dataset):
  def __init__(self,pointclouds,labels):
    self.pointclouds = pointclouds
    self.labels = labels

  def __len__(self):
      return len(self.pointclouds)

  def __getitem__(self,idx):
      return torch.tensor(self.pointclouds[idx], dtype=torch.float32),torch.tensor(self.labels[idx],dtype=torch.long)

dataset = PointCloudDataset(pointclouds,labels)
dataloader = DataLoader(dataset,batch_size=32,shuffle=True)

In [5]:
#pointnet model
import torch.nn as nn
import torch.nn.functional as F

class PointNet(nn.Module):
    def __init__(self, num_classes=4):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, 1, bias = True)
        self.conv2 = nn.Conv1d(64, 128, 1,bias = True )
        self.conv3 = nn.Conv1d(128, 256, 1,bias = True)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = torch.max(x, 4)[0]  # Global max pooling
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
#training the pointnet model
import torch.optim as optim
#parameters
epochs = 500
learning_rate = 0.001
num_classes = 4

#model, loss function and optimizer

model = PointNet(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#training loop
for epoch in range(epochs):
  model.train()
  total_loss = 0
  correct = 0
  total = 0

  for data in dataloader:
      points, labels = data
      points = points.transpose(1,2)
      optimizer.zero_grad()

      #forward pass
      outputs = model(points)
      loss = criterion(outputs, labels)

      #backward-pass and optimizatin
      loss.backward()
      optimizer.step()

      total_loss += loss.item()

      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss: .4f}, Accuracy: {accuracy: .2f}%")


IndexError: Dimension out of range (expected to be in range of [-3, 2], but got 4)

In [ ]:
#evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in dataloader:
        points, labels = data
        points = points.transpose(1,2)
        outputs = model(points)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the dataset: {accuracy:.2f}%')